# Random Forest (1)

- 의사결정트리를 사용하는 가장 대표적인 배경 모델
- 의사결정트리의 단점(i.e., 과적함이 자주 발생)을 보완하고 장점은 유지한다
- 다른 모델들보다 학습 시간이 오래 걸린다
- 최근 XGBoost, LightGBM, CatBoost와 함께 주목받는 알고리즘 중 하나이다

> 예제 진행을 위해 `imblearn` 패키지의 설치가 필요하다

## 1. 패키지 참조

In [27]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
import seaborn as sb
from pandas import read_excel, DataFrame
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score

# 데이터 불균형 해소를 위한 패키지
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

## 2. 데이터 가져오기
- 출처: kaggle, 분류예제
> https://www.kaggle.com/competitions/otto-group-product-classification-challenge/data


|필드이름|설명|
|--|--|
|target|타겟(종속)변수 `(Class_1 ~ Class_2)`|
|feat_1 ~ feat_93|설명(독립)변수|

In [30]:
origin = read_excel('https://data.hossam.kr/G02/otto_train.xlsx')
(origin.info())
origin

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61878 entries, 0 to 61877
Data columns (total 94 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feat_1   61878 non-null  int64 
 1   feat_2   61878 non-null  int64 
 2   feat_3   61878 non-null  int64 
 3   feat_4   61878 non-null  int64 
 4   feat_5   61878 non-null  int64 
 5   feat_6   61878 non-null  int64 
 6   feat_7   61878 non-null  int64 
 7   feat_8   61878 non-null  int64 
 8   feat_9   61878 non-null  int64 
 9   feat_10  61878 non-null  int64 
 10  feat_11  61878 non-null  int64 
 11  feat_12  61878 non-null  int64 
 12  feat_13  61878 non-null  int64 
 13  feat_14  61878 non-null  int64 
 14  feat_15  61878 non-null  int64 
 15  feat_16  61878 non-null  int64 
 16  feat_17  61878 non-null  int64 
 17  feat_18  61878 non-null  int64 
 18  feat_19  61878 non-null  int64 
 19  feat_20  61878 non-null  int64 
 20  feat_21  61878 non-null  int64 
 21  feat_22  61878 non-null  int64 
 22

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61873,1,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,2,0,Class_9
61874,4,0,0,0,0,0,0,0,0,0,...,0,2,0,0,2,0,0,1,0,Class_9
61875,0,0,0,0,0,0,0,3,1,0,...,0,3,1,0,0,0,0,0,0,Class_9
61876,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,3,10,0,Class_9


## 3. 데이터 전처리
### 1) 타겟변수 라벨링

In [31]:
origin['target'] = origin['target'].map({
    'Class_1':0,
    'Class_2':1,
    'Class_3':2,
    'Class_4':3,
    'Class_5':4,
    'Class_6':5,
    'Class_7':6,
    'Class_8':7,
    'Class_9':8,
})

origin['target'].value_counts()

target
1    16122
5    14135
7     8464
2     8004
8     4955
6     2839
4     2739
3     2691
0     1929
Name: count, dtype: int64

> 각 class별로 데이터 불균형이 보이므로 이에 대한 처리가 필요하다
> 
> 데이터 불균형 해소는 데이터 분리 한 다음에 하는 것 권장

### 2) 독립/종속변수 분리

In [32]:
x = origin.drop(['target'], axis=1)
y = origin['target']
x.shape, y.shape

((61878, 93), (61878,))

### 3) 데이터 표준화 
> 이 데이터셋은 필요 없어 보이므로 생략

### 4) 훈련/검증 데이터 분할

In [37]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.3, random_state = 777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((43314, 93), (18564, 93), (43314,), (18564,))

### 5) 데이터 불균형 해소
#### 1_ Under Sampling 방식 - Random Under Sampler
- 많은 비율을 차지하는 다수 집단에서 일부만 샘플링하는 방식
- 소수 집단의 데이터가 어느 정도 확보되었다고 여겨질 때, 다수 집단의 데이터를 줄여서 균형을 맞추는 방식
- 다수 집단의 유의미한 데이터를 손실할 수 있다는 단점이 있다
##### `sampling_strategy` 파라미터
|값|설명|
|--|--|
|`majority`|다수 클래스만 다시 샘플링|
|`not majority`|`다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`not minority`|`소수 아님` - 소수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`all`|모든 클래스를 다시 샘플링|
|`auto`|자동 처리|

In [38]:
undersampler = RandomUnderSampler(sampling_strategy = 'majority',
                                  random_state=777)
x_under, y_under = undersampler.fit_resample(x_train, y_train)
print(x_under.shape, y_under.shape)
y_under.value_counts().sort_index()

(33336, 93) (33336,)


target
0    1339
1    1339
2    5598
3    1864
4    1914
5    9896
6    1956
7    5941
8    3489
Name: count, dtype: int64

##### 2_ Over Sampling - Random Over Sampler
- 소수 집단에서 복원 추출을 수행함는 방법
- 언더 셈플링과 달리 데이터 손실은 발생하지 않지만, 동일한 데이터를 여러번 학습 데이터에 포함시키므로 학습 정확도는 높지만 과적합 리스크가 크다
##### `sampling_strategy` 파라미터
|값|설명|
|--|--|
|`mionority`| 소수 클래스만 다시 샘플링|
|`not majority`|`다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`not minority`|`소수 아님` - 소수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`all`|모든 클래스를 다시 샘플링|
|`auto`|자동 처리|

In [42]:
oversampler = RandomOverSampler(sampling_strategy = 'minority',
                                  random_state=777)
x_over, y_over = oversampler.fit_resample(x_train, y_train)
print(x_over.shape, y_over.shape)
y_over.value_counts().sort_index()

(53292, 93) (53292,)


target
0    11317
1    11317
2     5598
3     1864
4     1914
5     9896
6     1956
7     5941
8     3489
Name: count, dtype: int64

##### 3_ Over Sampling의 한 중류 - SMOTE
- 소수 집단의 데이터를 바탕으로 새로운 데이터를 생성
- 단순히 소수 집단의 데이터를 복원 추출하는 것이 아니라 소수 집단 데이터를 분석해 어떤 특징이 있는지 살피고 그와 유사한 패턴을 갖는 `가짜 데이터를 생성`한다

##### `sampling_strategy` 파라미터
|값|설명|
|--|--|
|`mionority`| 소수 클래스만 다시 샘플링|
|`not majority`|`다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`not minority`|`소수 아님` - 소수 클래스를 제외한 모든 클래스를 다시 샘플링|
|`all`|모든 클래스를 다시 샘플링|
|`auto`|자동 처리|

- 혹은 실수 타입으로 설정할 경우 샘플 수의 비율을 의미

##### `k_neighbors` 파라미터 (int)
- 합성 샘플을 생성하는데 사용할 샘플의 가장 가까운 이웃 수 (기본값 = 5)

In [44]:
# "sampling_strategy" can be a float only when the type of target is binary
smote_sampler = SMOTE(sampling_strategy='minority', random_state=777)
x_sm, y_sm = smote_sampler.fit_resample(x_train, y_train)
print(x_sm.shape, y_sm.shape)
y_sm.value_counts().sort_index()

(53292, 93) (53292,)


target
0    11317
1    11317
2     5598
3     1864
4     1914
5     9896
6     1956
7     5941
8     3489
Name: count, dtype: int64

## 3. Random Forest 모델 적합
### 1) 단일 모델 만들기

In [46]:
# 첫번째 시도
# rfc = RandomForestClassifier(n_estimators=20,max_depth=5,random_state=777)
# 두번째 시도
# rfc = RandomForestClassifier(n_estimators=50,max_depth=30,random_state=777)
# 세번째 시도
# rfc = RandomForestClassifier(n_estimators=100, max_depth=30,random_state=777)
rfc = RandomForestClassifier(n_estimators=100,       # 샘플 수
                             max_depth=100,          # 
                             random_state=777)

# 원본 데이터로 학습 진행
rfc.fit(x_train, y_train)
print('훈련 정확도: ', rfc.score(x_train, y_train))

# UnderSampling 데이터로 학습 진행
rfc.fit(x_under, y_under)
print('훈련 정확도: ', rfc.score(x_under, y_under))

# OverSampling 데이터로 학습 진행
rfc.fit(x_over, y_over)
print('훈련 정확도: ', rfc.score(x_over, y_over))

# SMOTE 데이터로 학습 진행
rfc.fit(x_sm, y_sm)
print('훈련 정확도: ', rfc.score(x_sm, y_sm))
print('검증 정확도: ', rfc.score(x_test, y_test))

훈련 정확도:  1.0
훈련 정확도:  1.0
훈련 정확도:  1.0
훈련 정확도:  0.9999437063724387
검증 정확도:  0.7989118724412843


### 2) 하이퍼파라미터 튜닝

In [47]:
rfc = RandomForestClassifier(random_state=777)
params = {
    'n_estimators': [20, 50, 100],
    'max_depth': [5, 30, 100]
}

grid = GridSearchCV(rfc, param_grid=params, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

print('최적의 하이퍼파라미터: ', grid.best_params_)
print('최대 훈련 정확도:', grid.best_score_)

y_pred = grid.best_estimator_.predict(x_test)
print('최대 검증 정확도:', accuracy_score(y_test, y_pred))

result_df = DataFrame(grid.cv_results_['params'])
result_df['mean_test_score'] = grid.cv_results_['mean_test_score']
result_df.sort_values(by='mean_test_score',ascending=False)

최적의 하이퍼파라미터:  {'max_depth': 100, 'n_estimators': 100}
최대 훈련 정확도: 0.7999030982674508
최대 검증 정확도: 0.807207498383969


,max_depth,n_estimators,mean_test_score
8,100,100,0.799903
7,100,50,0.796832
5,30,100,0.795747
4,30,50,0.792608
3,30,20,0.785820
6,100,20,0.785751
0,5,20,0.612758
2,5,100,0.606709
1,5,50,0.606571
